In [5]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [8]:
from google.cloud import bigquery

In [9]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,8.818490,False,17,1,42.0
1,8.141671,False,29,1,38.0
2,5.948072,True,38,1,38.0
3,8.838332,True,27,1,39.0
4,9.259415,True,28,1,38.0


In [10]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,9991.000000,10000.000000,10000.000000,9881.000000
mean,7.279259,27.332300,1.033200,38.704888
std,1.317603,6.238122,0.188948,2.602659
min,0.661387,12.000000,1.000000,19.000000
25%,6.624891,22.000000,1.000000,38.000000
50%,7.374463,27.000000,1.000000,39.000000
75%,8.124034,32.000000,1.000000,40.000000
max,12.808857,53.000000,3.000000,47.000000


In [11]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [12]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [13]:
data.head()

,is_male,mother_age,plurality,gestation_weeks
757,1,23,1,39.0
6544,1,19,1,38.0
3190,1,28,1,40.0
2064,0,32,1,40.0
4066,0,17,1,40.0


In [14]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [18]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)

In [19]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])

In [20]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 64)                320       
_________________________________________________________________
dense_10 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 33        
Total params: 2,433
Trainable params: 2,433
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

2021-10-11 04:56:56.150972: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
209/209 [==============================] - 2s 6ms/step - loss: 1.4645 - mae: 0.9497 - mse: 1.4645 - val_loss: 1.2881 - val_mae: 0.8793 - val_mse: 1.2881
Epoch 2/10
209/209 [==============================] - 1s 4ms/step - loss: 1.3527 - mae: 0.9176 - mse: 1.3527 - val_loss: 2.4411 - val_mae: 1.2929 - val_mse: 2.4411
Epoch 3/10
209/209 [==============================] - 1s 5ms/step - loss: 1.3397 - mae: 0.9093 - mse: 1.3397 - val_loss: 1.2575 - val_mae: 0.8721 - val_mse: 1.2575
Epoch 4/10
209/209 [==============================] - 1s 4ms/step - loss: 1.2921 - mae: 0.8926 - mse: 1.2921 - val_loss: 1.4663 - val_mae: 0.9572 - val_mse: 1.4663
Epoch 5/10
209/209 [==============================] - 1s 4ms/step - loss: 1.2792 - mae: 0.8899 - mse: 1.2792 - val_loss: 1.4568 - val_mae: 0.9377 - val_mse: 1.4568
Epoch 6/10
209/209 [==============================] - 1s 4ms/step - loss: 1.2612 - mae: 0.8873 - mse: 1.2612 - val_loss: 1.3944 - val_mae: 0.9201 - val_mse: 1.3944
Epoch 7/10
209/2

In [22]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])

In [23]:
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()

Predicted val:  7.263769
Actual val:  7.25100379718

Predicted val:  7.471038
Actual val:  7.2862777591

Predicted val:  6.616461
Actual val:  4.6848230675

Predicted val:  6.876293
Actual val:  10.460934331899999

Predicted val:  7.4202805
Actual val:  8.24969784404

Predicted val:  6.9501576
Actual val:  6.3493131456

Predicted val:  7.4446445
Actual val:  6.8122838958

Predicted val:  6.605333
Actual val:  7.978529261779999

Predicted val:  7.0495853
Actual val:  7.50012615324

Predicted val:  7.2001987
Actual val:  8.12623897732



In [38]:
wit_data = pd.concat([x_test, y_test], axis=1)

In [39]:
def custom_predict(examples_to_infer):
    preds = model.predict(examples_to_infer)
    return preds

In [40]:
try:
  import google.colab
  !pip install --upgrade witwidget
except:
  pass

In [41]:
from witwidget.notebook.visualization import WitConfigBuilder
from witwidget.notebook.visualization import WitWidget

In [42]:
config_builder = (WitConfigBuilder(wit_data[:500].values.tolist(), data.columns.tolist() + ['weight_pounds'])
  .set_custom_predict_fn(custom_predict)
  .set_target_feature('weight_pounds')
  .set_model_type('regression'))
WitWidget(config_builder, height=800)

WitWidget(config={'model_type': 'regression', 'label_vocab': [], 'feature_names': ['is_male', 'mother_age', 'p…